<a href="https://colab.research.google.com/github/fymatsushita/pediatrics/blob/main/PredictiveModel_24h_Mortality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lux

In [ ]:
df.head()

In [ ]:
df['Alert'] = df['Alert'].astype(float)
df.info()

In [ ]:
df.describe().T

In [ ]:
p = df.hist(figsize=(10,10))

In [ ]:
df.shape

In [ ]:
print(df.isnull().sum())

In [ ]:
l = sns.pairplot(df, hue='Alert')

In [ ]:
X_train = df.loc[:,df.columns != 'Alert']
y_train = df['Alert']

In [1]:
from BorutaShap import BorutaShap

feature_selector = BorutaShap(importance_measure='shap', classification=False)
feature_selector.fit(X=X_train, y=y_train, n_trials=50, random_state=0)

ModuleNotFoundError: ignored

In [ ]:
feature_selector.plot(which_features='all', figsize=(16,12))

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance

from sklearn.model_selection import train_test_split
X = df.loc[:,df.columns != 'Alert']
y = df['Alert']

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3)


In [ ]:
model_xgb = XGBClassifier()
model_xgb.fit(train_X, train_y, verbose=False)
xgb.plot_importance(model_xgb, max_num_features=10)
plt.show()

In [ ]:
pred_xgb = model_xgb.predict(test_X)

In [ ]:
# Grid search
def GridSearchModel(X, y, model, parameters, cv):
  CV_model = GridSearchCV(estimator=model, param_grid = parameters, cv = cv)
  CV_model.fit(X,y)
  CV_model.cv_results_
  print("Best Score:", CV_model.best_score_, " / Best parameters:", CV_model.best_params_)

In [ ]:
# Learning curve
def LearningCurve(X, y, model, cv, train_sizes):

    train_sizes, train_scores, test_scores = learning_curve(model, X, y, cv = cv, n_jobs = 4, 
                                                            train_sizes = train_sizes)

    train_scores_mean = np.mean(train_scores, axis = 1)
    train_scores_std  = np.std(train_scores, axis = 1)
    test_scores_mean  = np.mean(test_scores, axis = 1)
    test_scores_std   = np.std(test_scores, axis = 1)
    
    train_Error_mean = np.mean(1- train_scores, axis = 1)
    train_Error_std  = np.std(1 - train_scores, axis = 1)
    test_Error_mean  = np.mean(1 - test_scores, axis = 1)
    test_Error_std   = np.std(1 - test_scores, axis = 1)

    Scores_mean = np.mean(train_scores_mean)
    Scores_std = np.mean(train_scores_std)
    
    _, y_pred, Accuracy, Error, precision, recall, f1score = ApplyModel(X, y, model)
    
    plt.figure(figsize = (16,4))
    plt.subplot(1,2,1)
    ax1 = Confuse(y, y_pred, classes)
    plt.subplot(1,2,2)
    plt.fill_between(train_sizes, train_Error_mean - train_Error_std,train_Error_mean + train_Error_std, alpha = 0.1,
                     color = "r")
    plt.fill_between(train_sizes, test_Error_mean - test_Error_std, test_Error_mean + test_Error_std, alpha = 0.1, color = "g")
    plt.plot(train_sizes, train_Error_mean, 'o-', color = "r",label = "Training Error")
    plt.plot(train_sizes, test_Error_mean, 'o-', color = "g",label = "Cross-validation Error")
    plt.legend(loc = "best")
    plt.grid(True)
     
    return (model, Scores_mean, Scores_std )

In [ ]:
def ApplyModel(X, y, model):
    
    model.fit(X, y)
    y_pred  = model.predict(X)

    Accuracy = round(np.median(cross_val_score(model, X, y, cv = cv)),2)*100
 
    Error   = 1 - Accuracy
    
    precision = precision_score(y_train, y_pred) * 100
    recall = recall_score(y_train, y_pred) * 100
    f1score = f1_score(y_train, y_pred) * 100
    
    return (model, y_pred, Accuracy, Error, precision, recall, f1score) 

In [ ]:
def Confuse(y, y_pred, classes):
    cnf_matrix = confusion_matrix(y, y_pred)
    
    cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis = 1)[:, np.newaxis]
    c_train = pd.DataFrame(cnf_matrix, index = classes, columns = classes)  

    ax = sns.heatmap(c_train, annot = True, cmap = cmap, square = True, cbar = False, 
                          fmt = '.2f', annot_kws = {"size": 20})
    return(ax, c_train)

In [ ]:
def PrintResults(model, X, y, title):
    
    model, y_pred, Accuracy, Error, precision, recall, f1score = ApplyModel(X, y, model)
    
    _, Score_mean, Score_std = LearningCurve(X, y, model, cv, train_size)
    Score_mean, Score_std = Score_mean*100, Score_std*100
    
    
    print('Scoring Accuracy: %.2f %%'%(Accuracy))
    print('Scoring Mean: %.2f %%'%(Score_mean))
    print('Scoring Standard Deviation: %.4f %%'%(Score_std))
    print("Precision: %.2f %%"%(precision))
    print("Recall: %.2f %%"%(recall))
    print('f1-score: %.2f %%'%(f1score))
    
    Summary = pd.DataFrame({'Model': title,
                       'Accuracy': Accuracy, 
                       'Score Mean': Score_mean, 
                       'Score St Dv': Score_std, 
                       'Precision': precision, 
                       'Recall': recall, 
                       'F1-Score': f1score}, index = [0])
    return (model, Summary)

In [ ]:
from sklearn.model_selection import ShuffleSplit
classes = ['0','1']
cv = ShuffleSplit(n_splits = 100, test_size = 0.3, random_state = 0)
train_size = np.linspace(.1, 1.0, 15)

In [ ]:
from sklearn.model_selection import train_test_split 
X = df.drop(['pCO2', 'pH', 'BIC', 'DOL','Alert', 'Lactate','BE','cGA'], 1)
X2 = df.drop(['Alert'], 1)
y = df['Alert']

X_train, X_test, y_train, y_test = train_test_split (X, y, test_size=0.3, random_state=40, stratify=y)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
cmap = sns.diverging_palette(220,10, as_cmap=True)
model = LogisticRegression()
model, Summary_LR = PrintResults(model, X_train, y_train, 'Logistic Regression')

y_train_LR = pd.Series(model.predict(X_train), name = "LR")
y_test_LR = pd.Series(model.predict(X_test), name = "LR")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
%timeit
model = RandomForestClassifier(n_estimators = 200)
model,Summary_RF = PrintResults(model, X_train, y_train, 'Random Forest')
y_train_RF = pd.Series(model.predict(X_train), name="RF")
y_test_RF = pd.Series(model.predict(X_test), name = "RF")

In [ ]:
model, y_pred, Accuracy, Error, precision, recall, f1score = ApplyModel(X_train, y_train, model)
Priority = pd.DataFrame({'Feature': X_train.columns, 'Importance':np.round(model.feature_importances_,3)})
Priority = Priority.sort_values('Importance',ascending=False).set_index('Feature')
print(Priority)

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

X_scaled = StandardScaler().fit_transform(X)
X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = train_test_split (X_scaled, y, test_size=0.3, random_state=40, stratify=y)

model = SVC()
model, Summary_SVM = PrintResults(model, X_train_scaled, y_train_scaled, 'SVM')
y_train_SVM = pd.Series(model.predict(X_train_scaled), name = "SVM")
y_test_SVM = pd.Series(model.predict(X_test_scaled), name = 'SVM')

In [ ]:
from sklearn.svm import LinearSVC
model = LinearSVC()
model,Summary_LSVM = PrintResults(model, X_train_scaled, y_train,"LSVM")
y_train_LSVM = pd.Series(model.predict(X_train_scaled), name = "LSVM")
y_test_LSVM = pd.Series(model.predict(X_test_scaled), name = "LSVM")

In [ ]:
from sklearn.linear_model import SGDClassifier
model = SGDClassifier(max_iter = 200, tol = None)
model, Summary_SGD = PrintResults(model, X_train_scaled, y_train, 'SGD')
y_train_SGD = pd.Series(model.predict(X_train_scaled), name = "SGD")
y_test_SGD = pd.Series(model.predict(X_test_scaled), name= 'SGD')

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors = 3)
model, Summary_KNN = PrintResults(model, X_train_scaled, y_train, 'KNN')
y_train_KNN = pd.Series(model.predict(X_train_scaled), name = "KNN")
y_test_KNN = pd.Series(model.predict(X_test_scaled), name="KNN")

In [ ]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model,Summary_GNB = PrintResults(model, X_train, y_train, "GNB")
y_train_GNB = pd.Series(model.predict(X_train), name = "GNB")
y_test_GNB = pd.Series(model.predict(X_test), name = "GNB")

In [ ]:
from sklearn.linear_model import Perceptron
model = Perceptron(max_iter = 5)
model,Summary_MLP = PrintResults(model, X_train_scaled, y_train, 'MLP')
y_train_MLP = pd.Series(model.predict(X_train_scaled), name = "MLP")
y_test_MLP = pd.Series(model.predict(X_test_scaled), name = "MLP")

In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier(objective="binary:logistic", eta=0.01, max_depth=6, min_child_weight=1, 
                      scale_pos_weight=1, gamma=0, colsample_bytree=0.9, nthread=4,
                      early_stopping_round=5)
model,Summary_XGB = PrintResults(model, X_train, y_train, 'XGB')
y_train_XGB = pd.Series(model.predict(X_train), name = "XGB")
y_test_XGB = pd.Series(model.predict(X_test), name = "XGB")

In [ ]:
Class_Results = pd.concat([Summary_LR, Summary_SGD, Summary_RF, 
                           Summary_SVM, Summary_KNN, Summary_GNB,
                           Summary_MLP, Summary_LSVM, Summary_XGB], ignore_index = True)
    

Class_Results = Class_Results.sort_values(by = 'F1-Score', ascending=False)
Class_Results = Class_Results.set_index('F1-Score')
print(Class_Results)

In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt

plt.figure()

# Add the models to the list that you want to view on the ROC plot
models = [
{
    'label': 'Logistic Regression',
    'model': LogisticRegression(),
},
{
    'label': 'Random Forest',
    'model': RandomForestClassifier(),
},
{
    'label': 'XGBoost',
    'model': XGBClassifier()
},
{
    'label': 'Gaussian Naive Bayes',
    'model': GaussianNB()
}
]

# Below for loop iterates through your models list
for m in models:
    model = m['model'] # select the model
    model.fit(X_train, y_train) # train the model
    y_pred=model.predict(X_test) # predict the test data

# Compute False postive rate, and True positive rate
    pred = model.predict_proba(X_test)[:,1]
    fpr, tpr, thresholds = metrics.roc_curve(y_test, pred)
# Calculate Area under the curve to display on the plot
    auc = metrics.roc_auc_score(y_test,pred)
# Now, plot the computed values
    plt.plot(fpr, tpr, label='%s ROC (area = %0.2f)' % (m['label'], auc))
# Custom settings for the plot 
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1-Specificity(False Positive Rate)')
plt.ylabel('Sensitivity(True Positive Rate)')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()   # Display